In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
import tvm
from tvm import relay
from tvm.relay.dataflow_pattern import rewrite
from tvm.relay.dataflow_pattern import *
import numpy as np
from tvm.relay import transform
from tvm.relay.testing import run_opt_pass
from tvm.relay import transform, build_module
from tvm.relay import testing
import tvm.testing
# from SlicingMachine import TVMSlicer
import mozer
from mozer.slicer.SlicingMachine import TVMSlicer
from mozer.slicer.Quantize import quantize
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tvm
import tvm.relay as relay
from tvm.contrib import graph_executor 
import numpy as np
import json
import pygraphviz as pgv
from argparse import ArgumentParser
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *
from tvm import rpc
from tvm.autotvm.measure.measure_methods import set_cuda_target_arch
from tensorflow import keras
from tvm.contrib.download import download_testdata
import subprocess
from tvm.contrib import relay_viz
from tvm.relay import build_module

/home/jd/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def graph_split2(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    if (
                        split_conf
                        and split_operator_name in operator_index_map
                        and operator_index_map[split_operator_name] >= split_operator_index
                    ):
                        # Do graph splitting.
                        split_conf.pop(0)
                        snode_dep.append({"nodes": {}, "ref_nodes": {}})
                        ann = _recursion(
                            anf.body,
                            pipeline_mods,
                            split_conf,
                            constant_expr,
                        )
                        snode_dep.pop()
                        dep_vars = get_dep_var(snode_dep)
                        # When the nodes of the current subgraph are the depedency node of another
                        # subgraph, we need to set them as the output of current subgraph.
                        body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                        # when the operator of current subgraph uses previous subgraph constant
                        # as the argument of a "relay.expr.call", such constant may become a free
                        # varaible if the constant does not exist in the current subgraph.
                        # merge the previous constant with current subgraph to avoid such issue.
                        if constant_expr:
                            ann = merge_constant_expr(constant_expr, ann)
                        # ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                        # mod = tvm.IRModule.from_expr(ann)
                        pipeline_mods.insert(0, ann)
                        # Return the last node of the current subgraph.
                        return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf
    
    def getting_inputs(mod):
        name_hints = []
        ann = run_opt_pass(mod.body, transform.ToGraphNormalForm())
        mod = tvm.IRModule.from_expr(ann)
        for param in mod['main'].params:
            name_hints.append(param.name_hint)
        return name_hints

    def setting_outputs(anf, name_hints, outputs):
        # Get last element in snode_dep : current node's dependency
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                setting_outputs(anf.body, name_hints, outputs),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # print(anf.var.name_hint)
            if anf.var.name_hint in name_hints:
                outputs.append(anf)
            # value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                setting_outputs(anf.body, name_hints, outputs),
            )
        # Or End
        else:
            # new_map = 
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    # ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    # mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, ann.body)
    inputs = []
    for mod in pipeline_mods:
        inputs.extend(getting_inputs(mod))
    print(inputs)
    total_outputs = []
    for mod in pipeline_mods:
        outputs = []
        setting_outputs(mod, inputs, outputs)
        total_outputs.append(outputs)
    return pipeline_mods, total_outputs


In [3]:
def get_inputs(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                print(anf.var.name_hint)
                print(anf.var)
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    # split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    # split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    # if (
                    #     split_conf
                    #     and split_operator_name in operator_index_map
                    #     and operator_index_map[split_operator_name] >= split_operator_index
                    # ):
                    #     # Do graph splitting.
                    #     split_conf.pop(0)
                    #     snode_dep.append({"nodes": {}, "ref_nodes": {}})
                    #     ann = _recursion(
                    #         anf.body,
                    #         pipeline_mods,
                    #         split_conf,
                    #         constant_expr,
                    #     )
                    #     snode_dep.pop()
                    #     dep_vars = get_dep_var(snode_dep)
                    #     # When the nodes of the current subgraph are the depedency node of another
                    #     # subgraph, we need to set them as the output of current subgraph.
                    #     body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                    #     # when the operator of current subgraph uses previous subgraph constant
                    #     # as the argument of a "relay.expr.call", such constant may become a free
                    #     # varaible if the constant does not exist in the current subgraph.
                    #     # merge the previous constant with current subgraph to avoid such issue.
                    #     if constant_expr:
                    #         ann = merge_constant_expr(constant_expr, ann)
                    #     ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                    #     mod = tvm.IRModule.from_expr(ann)
                    #     pipeline_mods.insert(0, mod)
                    #     # Return the last node of the current subgraph.
                        # return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, mod)
    return pipeline_mods


In [32]:
def setting_outputs(anf, outputs):
    # Get last element in snode_dep : current node's dependency
    # If function -> decouple
    if isinstance(anf, tvm.relay.Function):
        return tvm.relay.Function(
            anf.params,
            setting_outputs(anf.body, outputs),
            anf.ret_type,
            anf.type_params,
            anf.attrs,
        )
    # Function of Let
    if isinstance(anf, tvm.relay.expr.Let):
        value = anf.value
        # print(anf.var.name_hint)
        # if anf.var.name_hint in name_hints:
            # outputs.append(anf)
        # value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
        return tvm.relay.expr.Let(
            anf.var,
            value,
            setting_outputs(anf.body, outputs),
        )
    # Or End
    else:
        new_outputs = []
        for o in outputs:
            new_outputs.append(o.var)
        # for o in outputs:
        #     new_outputs.append(tvm.relay.expr.Let(
        #             o.var,
        #             o.value,
        #             o.body,
        #     ))
        new_map = tvm.relay.expr.Tuple(new_outputs)
        return new_map

In [3]:
from tvm.relay.dataflow_pattern import *

In [34]:
class SetOutputCallback(DFPatternCallback):
    def __init__(self, name_hints=[],require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.pattern = wildcard()

    def callback(self, pre, post, node_map):
        try:
            print(type(pre))
            print(pre.name_hint)
        except:
            pass
        return pre 

In [2]:
base_path = "./simple_model.h5"
input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
model_keras = tf.keras.models.load_model(base_path)
# tvm result
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}


In [3]:
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [23]:
mod['main']

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(64, 32, 3, 3), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(96, 16, 3, 3), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(48, 1, 3, 3), float32], %v_param_12: Tensor[(1), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
 

In [45]:
type(upc0.match_node[0])

tvm.relay.expr.Call

In [31]:
a = upc0.match_node[0]

In [39]:
a.op.name

'nn.bias_add'

In [41]:
a.args[0].op.name

'nn.conv2d'

In [88]:
class UnetCallback(DFPatternCallback):
    """
    A callback class to rewrite the matched pattern to a batch_norm op.
    """
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        # self.tuple_get_item_node = is_tuple([wildcard(), self.x])
        # self.pattern_1 = self.tuple

        self.pattern = is_op(match_node[0].op.name)(wildcard(), wildcard())
        # self.pattern = is_op(match_node[0].op.name)
        self.match_node = match_node
        self.tmp = []

    def quant(self, node):
        # cast_to_int8 = relay.cast(
        #     relay.clip(
        #         relay.round(
        #             relay.multiply(node, relay.const(8.0))
        #         ), 
        #         a_min=-127.0, a_max=127.0
        #     ),
        #     dtype="int8"
        # )

        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(
                        relay.subtract(node, relay.const(18.0))
                        , relay.const(7.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        result_node = relay.annotation.stop_fusion(cast_to_int8)
        self.tmp.append(result_node)
        return result_node

    def dequant(self, node):
        # cast_to_float32 = relay.divide(
        #     relay.cast(node, dtype='float32'), relay.const(8.0)
        # )
        cast_to_float32 = relay.add(
                relay.divide(
                relay.cast(node, dtype='float32'), relay.const(7.0))
            , relay.const(18.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern.match(pre):
            # print(type(post))
            # # return post
            if pre in self.match_node:
                print("sd")
                return self.dequant(self.quant(post))
        return post

In [5]:
upc0 = mozer.slicer.Quantize.UnetPreProcessCallback()
rewrite(upc0, mod['main'])

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(64, 32, 3, 3), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(96, 16, 3, 3), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(48, 1, 3, 3), float32], %v_param_12: Tensor[(1), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
 

In [89]:
upc1 = UnetCallback(upc0.match_node)
rewrite(upc1, mod['main'])

sd
sd


fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(64, 32, 3, 3), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(96, 16, 3, 3), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(48, 1, 3, 3), float32], %v_param_12: Tensor[(1), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = subtract(%4, 18f);
  %6 = multiply(%5, 7f);
  %7 = round(%6);
  %8 = clip(%7, a_min=-127

In [55]:
upc0.match_node[0].op.name

'nn.bias_add'

In [54]:
is_expr(upc0.match_node[0]) 

CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x31c760d8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x31c733c8), [])], relay.attrs.MaxPool2DAttrs(0x4244f28), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x31caee98), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x4623bb8), [])], relay.attrs.MaxPool2DAttrs(0x45ffa08), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x31caf398), []), Var(_param_6, ty=TensorType([64], float32))], relay.attrs.BiasAddAttrs(0x31caa5b8), [])

In [4]:
quantize(mod, 1)

2
2
[CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x318520d8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x31825598), [])], relay.attrs.MaxPool2DAttrs(0x41d28c8), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x31825698), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x419fa08), [])], relay.attrs.MaxPool2DAttrs(0x31858fb8), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x31843a18), []), Var(_param_6, ty=TensorType([64], float

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(64, 32, 3, 3), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(96, 16, 3, 3), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(48, 1, 3, 3), float32], %v_param_12: Tensor[(1), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
 

In [5]:
upc = mozer.slicer.Quantize.UnetPreProcessCallback()

In [6]:
out = rewrite(upc, mod['main'])

In [7]:
upc.match_node

[CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])], relay.attrs.MaxPool2DAttrs(0x325adf68), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4f00318), []), Var(_param_6, ty=TensorType([64], float32))], relay.attrs.BiasAddAttrs(0x325a33c8), []),
 CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d),

In [60]:
len(upc.match_node2)

2

In [55]:
concatenate_nodes = []
parent_nodes = []

def collector(mod):
    """Count the number of occurrences of each operator in the module"""
    ret = {}
    concatenate_nodes = []
    parent_nodes = []
    def visit(node):
        global node2

        var2 = wildcard()
        tuple_node = is_tuple([wildcard(),var2])
        concat_node = is_op('concatenate')(tuple_node)
        pattern = concat_node
        if pattern.match(node):
            concatenate_nodes.append(node)
            parent_nodes.append(node.args[0].fields[1])
            print(type(node))
            print(node.op.name)
            # op_name = node.op.name

    relay.analysis.post_order_visit(mod["main"], visit)
    return ret

In [72]:
collector(mod)

<class 'tvm.relay.expr.Call'>
concatenate
<class 'tvm.relay.expr.Call'>
concatenate


{}

In [61]:
len(parent_nodes)

2

In [73]:
parent_nodes == upc.match_node

True

In [70]:
concatenate_nodes == upc.match_node2

True

In [69]:
upc.match_node2[0]

CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])], relay.attrs.MaxPool2DAttrs(0x325adf68), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4f00318), []), Var(_param_6, ty=TensorType([64], float32))],

In [62]:
concatenate_nodes == upc.match_node

False

In [37]:
node2

CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])], relay.attrs.MaxPool2DAttrs(0x325adf68), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], fl

In [41]:
node2.op, node2.args

(Op(concatenate),
 [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])], relay.attrs.MaxPool2DAttrs(0x325adf68), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))

In [45]:
tu = node2.args[0]

In [51]:
tu.fields[1]

CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])

In [50]:
tu.fields[0]

CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x4eeded8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x325a3be8), [])], relay.attrs.MaxPool2DAttrs(0x4effe88), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x3254c718), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x3261bd58), [])], relay.attrs.MaxPool2DAttrs(0x325adf68), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(

In [5]:
out = quantize(mod, 2)

In [6]:
out

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(64, 32, 3, 3), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(96, 16, 3, 3), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(48, 1, 3, 3), float32], %v_param_12: Tensor[(1), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]);
  %3 = subtract(%2, 18f);
  %4 = multiply(%3, 7f);
  %5 = round(%4);
  %6 = clip(%5, a_min=-127f, a_max=127f);
  %7 = cast(%6, dtype="int8");
  %8 = annotation.stop_fusion(%7);
  %9 = cast(%8, dtype="float32");
  %10 = divide(%9, 7f

In [7]:
split_config = [{"op_name": "annotation.stop_fusion", "op_index": 0}]
# print(mod['main'])
subgraphs, outputs = graph_split2(out, split_config, params)

['input_1', 'x_0', 'data_n_0', 'x_8']


In [8]:
len(subgraphs)

2

In [12]:
print(subgraphs[0].astext(False))

#[version = "0.0.5"]
let %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */;
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %x_0, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(16), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_1, %x_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i

In [13]:
print(subgraphs[1].astext(False))

#[version = "0.0.5"]
let %x_23: float32 /* ty=float32 */ = 7f /* ty=float32 */;
let %x_21: float32 /* ty=float32 */ = 18f /* ty=float32 */;
let %x_17: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(32), float32] */;
let %x_15: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_12: Tensor[(64), float32] /* ty=Tensor[(64), float32] */ = meta[relay.Constant][2] /* ty=Tensor[(64), float32] */;
let %x_10: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][3] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][4] /* ty=Tensor[(32), float32] */;
let %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */ = meta[relay.Constant][5] /* ty=Tensor[(32, 16, 3, 3), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Te

In [20]:
aa = subgraphs[1]

In [21]:
aa.free_vars()

AttributeError: <class 'tvm.relay.expr.Let'> has no attribute free_vars

In [17]:
print(subgraphs[1].astext(False))

#[version = "0.0.5"]
let %x_17: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(32), float32] */;
let %x_15: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_12: Tensor[(64), float32] /* ty=Tensor[(64), float32] */ = meta[relay.Constant][2] /* ty=Tensor[(64), float32] */;
let %x_10: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][3] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][4] /* ty=Tensor[(32), float32] */;
let %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */ = meta[relay.Constant][5] /* ty=Tensor[(32, 16, 3, 3), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][6] /* ty=Tensor[(16), float32] */;
let %x_0: Tensor[(16, 3, 3, 3), float3

In [15]:
[o.var.name_hint for o in outputs[1]]

['x_0']

In [10]:
outputs[0][0].var.name_hint

'x_0'

In [24]:
print(subgraphs[0].astext(False))

#[version = "0.0.5"]
let %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */;
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %x_0, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(16), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_1, %x_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i

In [20]:
len(outputs[0][1:])

2

In [18]:
ll = [outputs[0][1:]]
len(ll)

1

In [33]:
modmod = setting_outputs(subgraphs[0], outputs[0][1:])

In [34]:
modmod

LetNode(Var(x_0, ty=TensorType([16, 3, 3, 3], float32)), Constant([[[[ 3.23163867e-02 -8.54633600e-02  5.41195720e-02]
   [-1.10573828e-01  1.81553543e-01 -1.22144818e-04]
   [ 2.55260766e-02  7.36784637e-02  1.07363641e-01]]

  [[ 2.55233049e-02 -4.54409420e-02  1.67028517e-01]
   [ 1.45357460e-01 -1.00949802e-01  7.94526637e-02]
   [-1.49513945e-01  1.69569105e-02 -5.45993596e-02]]

  [[-1.71270907e-01 -1.04798146e-01 -7.64890388e-02]
   [-3.08729708e-03  8.45972598e-02 -3.27159166e-02]
   [-1.12808652e-01 -7.80549943e-02  1.27093047e-02]]]


 [[[-1.79601535e-01 -3.15128714e-02  3.91178727e-02]
   [ 1.68302536e-01 -1.86983153e-01 -1.29431516e-01]
   [ 7.07907379e-02 -5.24235666e-02 -2.14526057e-02]]

  [[-1.49425566e-01  1.06974870e-01  1.21452183e-01]
   [-3.09053659e-02  1.61907017e-01  1.01942331e-01]
   [-1.82749078e-01 -1.50260255e-01 -9.02489349e-02]]

  [[ 1.36263520e-02  5.32492995e-03  6.63298666e-02]
   [-7.51340613e-02 -1.39353558e-01  7.36561418e-02]
   [ 9.17883217e-02 -

In [36]:
ann = run_opt_pass(modmod, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']


In [35]:
print(modmod.astext(False))

#[version = "0.0.5"]
let %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */;
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %x_0, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(16), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_1, %x_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i

In [11]:
print(subgraphs[0].astext(False))

#[version = "0.0.5"]
let %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */;
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %x_0, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(16), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_1, %x_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i

In [38]:
out

fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
  %3 = nn.conv2d(%2, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %4 = nn.bias_add(%3, meta[relay.Constant][3] /* ty=Tensor[(32), float32] */) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 3

In [8]:
print([o.var for o in outputs[0]])

[Var(x_0, ty=TensorType([16, 3, 3, 3], float32)), Var(x_8, ty=TensorType([1, 32, 128, 128], float32)), Var(x_20, ty=TensorType([1, 96, 64, 64], float32))]


In [13]:
prev = subgraphs[0]

In [9]:
print(subgraphs[0].astext(False))

#[version = "0.0.5"]
let %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */;
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %x_0, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(16), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_1, %x_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i

In [14]:
while True:
    try:
        prev = prev.body
        print(type(prev), prev.var)
    except:
        print(type(prev), prev)
        break

<class 'tvm.relay.expr.Let'> free_var %x_1: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */;
%x_1
<class 'tvm.relay.expr.Let'> free_var %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
%x_2
<class 'tvm.relay.expr.Let'> free_var %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */;
%x_3
<class 'tvm.relay.expr.Let'> free_var %x_4: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
%x_4
<class 'tvm.relay.expr.Let'> free_var %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */;
%x_5
<class 'tvm.relay.expr.Let'> free_var %x_6: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */;
%x_6
<class 'tvm.relay.expr.Let'> free_var %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */;
%x_7
<class 'tvm.relay.expr.Let'> free_var %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */;
%x_8
<class 'tvm.re

In [15]:
prev

Var(x_20, ty=TensorType([1, 96, 64, 64], float32))

In [6]:
outputs[0][1].var

Var(x_8, ty=TensorType([1, 32, 128, 128], float32))

In [22]:
a = relay.Tuple([outputs[0][1], subgraphs[0]])

In [23]:
a

Tuple([LetNode(Var(x_8, ty=TensorType([1, 32, 128, 128], float32)), CallNode(Op(nn.bias_add), [Var(x_6, ty=TensorType([1, 32, 128, 128], float32)), Var(x_7, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x31851c78), [TensorType([1, 32, 128, 128], float32), TensorType([32], float32)]), LetNode(Var(x_9, ty=TensorType([1, 32, (int64)64, (int64)64], float32)), CallNode(Op(nn.max_pool2d), [Var(x_8, ty=TensorType([1, 32, 128, 128], float32))], relay.attrs.MaxPool2DAttrs(0x31814f48), [TensorType([1, 32, 128, 128], float32)]), LetNode(Var(x_10, ty=TensorType([64, 32, 3, 3], float32)), Constant([[[[-0.00034386 -0.00349673  0.07773861]
   [-0.03190579 -0.01584832  0.02670189]
   [ 0.01921258 -0.07818282  0.03479443]]

  [[-0.00130421 -0.01830789  0.05642799]
   [-0.0170652  -0.02210277  0.04254849]
   [ 0.01875039 -0.08010064 -0.08040661]]

  [[ 0.02165145  0.0240651  -0.04487719]
   [-0.02167934 -0.00768209  0.0794398 ]
   [-0.02068903  0.06871933  0.07726983]]

  ...

  [[-0.0320555

In [9]:
print(subgraphs[1].astext(False))

#[version = "0.0.5"]
let %x_17: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(32), float32] */;
let %x_15: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_12: Tensor[(64), float32] /* ty=Tensor[(64), float32] */ = meta[relay.Constant][2] /* ty=Tensor[(64), float32] */;
let %x_10: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][3] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][4] /* ty=Tensor[(32), float32] */;
let %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */ = meta[relay.Constant][5] /* ty=Tensor[(32, 16, 3, 3), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][6] /* ty=Tensor[(16), float32] */;
let %x_0: Tensor[(16, 3, 3, 3), float3

In [10]:
prev = subgraphs[1]
for i in range(12):
    print(prev.body.var)
    prev = prev.body

free_var %x_15: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */;
%x_15
free_var %x_12: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
%x_12
free_var %x_10: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */;
%x_10
free_var %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */;
%x_7
free_var %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */;
%x_5
free_var %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
%x_2
free_var %x_0: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
%x_0
free_var %x_21: Tensor[(96, 16, 3, 3), float32] /* ty=Tensor[(96, 16, 3, 3), float32] */;
%x_21
free_var %x_22: Tensor[(1, 16, 128, 128), float32] /* ty=Tensor[(1, 16, 128, 128), float32] */;
%x_22
free_var %x_23: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
%x_23
free_var %x_24: Tensor[(1, 16, 128, 128), float32] /* ty=Tensor[(1, 16, 128, 128), float32] */;
%x_24
free_var %x_25: (Tenso

In [14]:
print(subgraphs[1].astext(False))

#[version = "0.0.5"]
let %x_17: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(32), float32] */;
let %x_15: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][1] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_12: Tensor[(64), float32] /* ty=Tensor[(64), float32] */ = meta[relay.Constant][2] /* ty=Tensor[(64), float32] */;
let %x_10: Tensor[(64, 32, 3, 3), float32] /* ty=Tensor[(64, 32, 3, 3), float32] */ = meta[relay.Constant][3] /* ty=Tensor[(64, 32, 3, 3), float32] */;
let %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */ = meta[relay.Constant][4] /* ty=Tensor[(32), float32] */;
let %x_5: Tensor[(32, 16, 3, 3), float32] /* ty=Tensor[(32, 16, 3, 3), float32] */ = meta[relay.Constant][5] /* ty=Tensor[(32, 16, 3, 3), float32] */;
let %x_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */ = meta[relay.Constant][6] /* ty=Tensor[(16), float32] */;
let %x_0: Tensor[(16, 3, 3, 3), float3

In [24]:
ann = run_opt_pass(a, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_6 is bound more than once, this is not valid IR
[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_7 is bound more than once, this is not valid IR
[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_8 is bound more than once, this is not valid IR
[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_9 is bound more than once, this is not valid IR
[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_10 is bound more than once, this is not valid IR
[13:46:51] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_11 

TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  4: tvm::IRModule

In [15]:
ann = run_opt_pass(subgraphs[1], transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

In [16]:
out

fn (%x_20: Tensor[(1, 96, 64, 64), float32] /* ty=Tensor[(1, 96, 64, 64), float32] */, %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */) {
  %0 = nn.conv2d_transpose(%x_20, meta[relay.Constant][0] /* ty=Tensor[(96, 16, 3, 3), float32] */, channels=16, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %2 = (%1, %x_8) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */;
  %3 = concatenate(%2, axis=1) /* ty=Tensor[(1, 48, 128, 128), float32] */;
  %4 = nn.conv2d_transpose(%3, meta[relay.Constant][2] /* ty=Tensor[(48, 1, 3, 3), float32] */, channels=1, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 1, 256, 256), float32] */;
  nn.bias_add(%4, meta[relay.C

In [13]:
print(prev.astext(False))

#[version = "0.0.5"]
free_var %x_24: Tensor[(1, 16, 128, 128), float32] /* ty=Tensor[(1, 16, 128, 128), float32] */;
free_var %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */;
let %x_25: (Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */ = (%x_24, %x_8) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */;
let %x_26: Tensor[(1, 48, 128, 128), float32] /* ty=Tensor[(1, 48, 128, 128), float32] */ = concatenate(%x_25, axis=1) /* ty=Tensor[(1, 48, 128, 128), float32] */;
let %x_27: Tensor[(48, 1, 3, 3), float32] /* ty=Tensor[(48, 1, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(48, 1, 3, 3), float32] */;
let %x_28: Tensor[(1, 1, 256, 256), float32] /* ty=Tensor[(1, 1, 256, 256), float32] */ = nn.conv2d_transpose(%x_26, %x_27, channels=1, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kern

In [33]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

In [11]:
tvm.IRModule.from_expr(run_opt_pass(subgraphs[0], transform.ToGraphNormalForm()))['main']


fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
  %3 = nn.conv2d(%2, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %4 = nn.bias_add(%3, meta[relay.Constant][3] /* ty=Tensor[(32), float32] */) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 3

In [12]:
tvm.IRModule.from_expr(run_opt_pass(subgraphs[1], transform.ToGraphNormalForm()))['main']


fn (%x_20: Tensor[(1, 96, 64, 64), float32] /* ty=Tensor[(1, 96, 64, 64), float32] */, %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */) {
  %0 = nn.conv2d_transpose(%x_20, meta[relay.Constant][0] /* ty=Tensor[(96, 16, 3, 3), float32] */, channels=16, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %2 = (%1, %x_8) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */;
  %3 = concatenate(%2, axis=1) /* ty=Tensor[(1, 48, 128, 128), float32] */;
  %4 = nn.conv2d_transpose(%3, meta[relay.Constant][2] /* ty=Tensor[(48, 1, 3, 3), float32] */, channels=1, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 1, 256, 256), float32] */;
  nn.bias_add(%4, meta[relay.C

In [28]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
# outs = outputs[0][1]
outs = run_opt_pass(outputs[0][1], transform.ToGraphNormalForm())
outs = tvm.IRModule.from_expr(outs)['main']

# outnew = run_opt_pass(out_1, transform.ToGraphNormalForm())
# outnew = tvm.IRModule.from_expr(outnew)['main']

# out1 = relay.Function(params, relay.Tuple([outs, out]))

In [29]:
tvm.IRModule.from_expr([out, outs])

TVMError: Traceback (most recent call last):
  1: TVMFuncCall
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  2: TVMFuncCall
 

In [27]:
outs

fn (%x_6: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */, %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */) {
  %0 = nn.bias_add(%x_6, %x_7) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %1 = nn.max_pool2d(%0, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 32, 64i64, 64i64), float32] */;
  %2 = nn.conv2d(%1, meta[relay.Constant][0] /* ty=Tensor[(64, 32, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=64, kernel_size=[3, 3]) /* ty=Tensor[(1, 64, 64, 64), float32] */;
  %3 = nn.bias_add(%2, meta[relay.Constant][1] /* ty=Tensor[(64), float32] */) /* ty=Tensor[(1, 64, 64, 64), float32] */;
  %4 = nn.max_pool2d(%3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 64, 32i64, 32i64), float32] */;
  %5 = nn.conv2d_transpose(%4, meta[relay.Constant][2] /* ty=Tensor[(64, 32, 3, 3), float32] */, channels=32, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i

In [22]:
out1

fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %20 = fn (%x_6: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */, %x_7: Tensor[(32), float32] /* ty=Tensor[(32), float32] */) {
    %0 = nn.bias_add(%x_6, %x_7) /* ty=Tensor[(1, 32, 128, 128), float32] */;
    %1 = nn.max_pool2d(%0, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 32, 64i64, 64i64), float32] */;
    %2 = nn.conv2d(%1, meta[relay.Constant][0] /* ty=Tensor[(64, 32, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=64, kernel_size=[3, 3]) /* ty=Tensor[(1, 64, 64, 64), float32] */;
    %3 = nn.bias_add(%2, meta[relay.Constant][1] /* ty=Tensor[(64), float32] */) /* ty=Tensor[(1, 64, 64, 64), float32] */;
    %4 = nn.max_pool2d(%3, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 64, 32i64, 32i64), float32] */;
    %5 = nn.conv2d_transpose(%4, meta[relay.Constant][2] 

In [18]:
outputs[0][1:]

[LetNode(Var(x_8, ty=TensorType([1, 32, 128, 128], float32)), CallNode(Op(nn.bias_add), [Var(x_6, ty=TensorType([1, 32, 128, 128], float32)), Var(x_7, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x2e6abf8), [TensorType([1, 32, 128, 128], float32), TensorType([32], float32)]), LetNode(Var(x_9, ty=TensorType([1, 32, (int64)64, (int64)64], float32)), CallNode(Op(nn.max_pool2d), [Var(x_8, ty=TensorType([1, 32, 128, 128], float32))], relay.attrs.MaxPool2DAttrs(0x3205c728), [TensorType([1, 32, 128, 128], float32)]), LetNode(Var(x_10, ty=TensorType([64, 32, 3, 3], float32)), Constant([[[[-0.00034386 -0.00349673  0.07773861]
    [-0.03190579 -0.01584832  0.02670189]
    [ 0.01921258 -0.07818282  0.03479443]]
 
   [[-0.00130421 -0.01830789  0.05642799]
    [-0.0170652  -0.02210277  0.04254849]
    [ 0.01875039 -0.08010064 -0.08040661]]
 
   [[ 0.02165145  0.0240651  -0.04487719]
    [-0.02167934 -0.00768209  0.0794398 ]
    [-0.02068903  0.06871933  0.07726983]]
 
   ...
 
   [[-0.

In [52]:
ann = run_opt_pass(subgraph, transform.ToGraphNormalForm())
submod = tvm.IRModule.from_expr(ann)['main']

In [44]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
outnew = run_opt_pass(out_1, transform.ToGraphNormalForm())
outnew = tvm.IRModule.from_expr(outnew)['main']

out1 = relay.Function(out.params, relay.Tuple([outnew, out]), out.ret_type, out.type_params, out.attrs)

In [26]:
rewrite(socb, out.body)

CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Constant([[[[ 3.23163867e-02 -8.54633600e-02  5.41195720e-02]
   [-1.10573828e-01  1.81553543e-01 -1.22144818e-04]
   [ 2.55260766e-02  7.36784637e-02  1.07363641e-01]]

  [[ 2.55233049e-02 -4.54409420e-02  1.67028517e-01]
   [ 1.45357460e-01 -1.00949802e-01  7.94526637e-02]
   [-1.49513945e-01  1.69569105e-02 -5.45993596e-02]]

  [[-1.71270907e-01 -1.04798146e-01 -7.64890388e-02]
   [-3.08729708e-03  8.45972598e-02 -3.27159166e-02]
   [-1.12808652e-01 -7.80549943e-02  1.27093047e-02]]]


 [[[-1.79601535e-01 -3.15128714e-02  3.91178727e-02]
   [ 1.68302536e-01 -1.86983153e-01 -1.29431516e-01]
   [ 

In [16]:
for i in subgraphs:
    print(type(i))

<class 'tvm.relay.function.Function'>
<class 'tvm.relay.expr.Let'>


In [41]:
outputs[0][0].var.name_hint

'x_0'

In [33]:
out = relay.Tuple(outputs[0])
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_8 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_9 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_10 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_11 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_12 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_1

TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  4: tvm::IRModule

In [31]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
# outnew = [run_opt_pass(i, transform.ToGraphNormalForm()) for i in outputs[0]]
out1 = relay.Function(out.params, relay.Tuple(outnew), out.ret_type, out.type_params, out.attrs)
mod = tvm.IRModule()
mod["main"] = out1

# with tvm.transform.PassContext(opt_level=4):
#     lib = relay.build(out1, 'cuda', params=params)

TVMError: Traceback (most recent call last):
  3: TVMFuncCall
  2: tvm::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const [clone .isra.0]
  1: tvm::IRModuleNode::Add(tvm::GlobalVar const&, tvm::BaseFunc const&, bool)
  0: tvm::WarnIfMalformed(tvm::IRModule const&, tvm::relay::Function)
  File "/home/jd/workspace/tvm-v0.9.0/src/ir/module.cc", line 191
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: fv.size() == 0 (3 vs. 0) : Function:
fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3])

In [27]:
# out = subgraphs[1]
# ann = run_opt_pass(out, transform.ToGraphNormalForm())
# out = tvm.IRModule.from_expr(ann)['main']
# out2 = relay.Function(out.params, out, out.ret_type, out.type_params, out.attrs)
# with tvm.transform.PassContext(opt_level=4):
#     lib = relay.build(out2, 'cuda', params=params)

[23:54:16] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_20 is bound more than once, this is not valid IR
[23:54:16] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_8 is bound more than once, this is not valid IR


TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  4: tvm::IRModule

In [19]:
out

fn (%x_20: Tensor[(1, 96, 64, 64), float32] /* ty=Tensor[(1, 96, 64, 64), float32] */, %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */) {
  %0 = nn.conv2d_transpose(%x_20, meta[relay.Constant][0] /* ty=Tensor[(96, 16, 3, 3), float32] */, channels=16, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %2 = (%1, %x_8) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */;
  %3 = concatenate(%2, axis=1) /* ty=Tensor[(1, 48, 128, 128), float32] */;
  %4 = nn.conv2d_transpose(%3, meta[relay.Constant][2] /* ty=Tensor[(48, 1, 3, 3), float32] */, channels=1, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 1, 256, 256), float32] */;
  nn.bias_add(%4, meta[relay.C

In [11]:
type(subgraphs[1])

tvm.relay.function.Function

In [21]:

ann = run_opt_pass(subgraphs.body, transform.ToGraphNormalForm())
tmp_mod = tvm.IRModule.from_expr(ann)

[[LetNode(Var(x_225, ty=TensorType([1, 32, 128, 128], float32)), CallNode(Op(nn.bias_add), [Var(x_223, ty=TensorType([1, 32, 128, 128], float32)), Var(x_224, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x30b172c8), [TensorType([1, 32, 128, 128], float32), TensorType([32], float32)]), LetNode(Var(x_226, ty=TensorType([1, 32, (int64)64, (int64)64], float32)), CallNode(Op(nn.max_pool2d), [Var(x_225, ty=TensorType([1, 32, 128, 128], float32))], relay.attrs.MaxPool2DAttrs(0x30aedf08), [TensorType([1, 32, 128, 128], float32)]), LetNode(Var(x_227, ty=TensorType([64, 32, 3, 3], float32)), Constant([[[[-0.00034386 -0.00349673  0.07773861]
     [-0.03190579 -0.01584832  0.02670189]
     [ 0.01921258 -0.07818282  0.03479443]]
  
    [[-0.00130421 -0.01830789  0.05642799]
     [-0.0170652  -0.02210277  0.04254849]
     [ 0.01875039 -0.08010064 -0.08040661]]
  
    [[ 0.02165145  0.0240651  -0.04487719]
     [-0.02167934 -0.00768209  0.0794398 ]
     [-0.02068903  0.06871933  0.0772698

In [20]:
subgraphs[1].body.body.body.body.body.body.body.body.body.body.body.body.body.body.body.body

LetNode(Var(x_29, ty=TensorType([1], float32)), Constant([0.]), LetNode(Var(x_30, ty=TensorType([1, 1, 256, 256], float32)), CallNode(Op(nn.bias_add), [Var(x_28, ty=TensorType([1, 1, 256, 256], float32)), Var(x_29, ty=TensorType([1], float32))], relay.attrs.BiasAddAttrs(0x309037a8), [TensorType([1, 1, 256, 256], float32), TensorType([1], float32)]), Var(x_30, ty=TensorType([1, 1, 256, 256], float32))))

In [13]:
# ann = run_opt_pass(subgraphs[1].body, transform.ToGraphNormalForm())
# mod = tvm.IRModule.from_expr(ann)

In [16]:
mod['main'].params[0].name_hint

'x_20'

In [5]:
out = quantize(mod, 2)
dev = tvm.cuda(0)
target = 'cuda'
# with tvm.transform.PassContext(opt_level=4, disabled_pass={}):
#     lib = relay.build(out, target, params=params)

In [6]:
out.body

Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(add), [CallNode(Op(divide), [CallNode(Op(cast), [CallNode(Op(annotation.stop_fusion), [CallNode(Op(cast), [CallNode(Op(clip), [CallNode(Op(round), [CallNode(Op(multiply), [CallNode(Op(subtract), [CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(add), [CallNode(Op(divide), [CallNode(Op(cast), [CallNode(Op(annotation.stop_fusion), [CallNode(Op(cast), [CallNode(Op(clip), [CallNode(Op(round), [CallNode(Op(multiply), [CallNode(Op(subtract), [CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(add), [CallNode(Op(divide), [CallNode(Op(cast), [CallNode(Op(annotation.stop_fusion), [CallNode(Op(cast), [CallNode(Op(clip), [CallNode(Op(round), [CallNode(Op(multiply), [CallNode(Op(subtract), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(add), [CallNode(Op(div

In [17]:
body = out.body
body = run_opt_pass(body, transform.ToANormalForm())
body = run_opt_pass(body, transform.InferType())

In [40]:
mod = tvm.IRModule()
mod['main'] = out

In [19]:
type(out)

tvm.relay.function.Function

In [6]:
submods = []
for g in subgraphs:
    ann = run_opt_pass(g.body, transform.ToGraphNormalForm())
    tmp_mod = tvm.IRModule.from_expr(ann)
    submods.append(tmp_mod)

In [10]:
mod = tvm.IRModule()
mod['main'] = out

In [11]:
graph_attr = {"color": "red"}
node_attr = {"color": "blue"}
edge_attr = {"color": "black"}
def get_node_attr(node):
    if "nn.conv2d" in node.type_name and "NCHW" in node.detail:
        return {
            "fillcolor": "green",
            "style": "filled",
            "shape": "box",
        }
    if "Var" in node.type_name:
        return {"shape": "ellipse"}
    return {"shape": "box"}
    
dot_plotter = relay_viz.DotPlotter(
    graph_attr=graph_attr,
    node_attr=node_attr,
    edge_attr=edge_attr,
    get_node_attr=get_node_attr)

viz = relay_viz.RelayVisualizer(
    # submods[1],
    mod,
    relay_param=params,
    plotter=dot_plotter,
    parser=relay_viz.DotVizParser())
viz.render("newnew")

# viz = relay_viz.RelayVisualizer(
#     # submods[0],
#     out1,
#     relay_param=params,
#     plotter=dot_plotter,
#     parser=relay_viz.DotVizParser())
# viz.render("simple_0")
# viz = relay_viz.RelayVisualizer(
#     mod,
#     relay_param=params,
#     plotter=dot_plotter,
#     parser=relay_viz.DotVizParser())
# viz.render("simple_total")

In [43]:
print(mod['main'].body.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
%0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
%1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
%2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
%3 = nn.conv2d(%2, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 128, 128), float32] */;
%4 = nn.bias_add(%3, meta[relay.Constant][3] /* ty=Tensor[(32), float32] */) /* ty=Tensor[(1, 32, 128, 128), float32] */;
%5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=

In [13]:
print(body)

free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
free_var %v_param_1: Tensor[(64, 3, 3, 3), float32] /* ty=Tensor[(64, 3, 3, 3), float32] */;
let %x_1745: Tensor[(1, 64, 256, 256), float32] /* ty=Tensor[(1, 64, 256, 256), float32] */ = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=64, kernel_size=[3, 3]) /* ty=Tensor[(1, 64, 256, 256), float32] */;
free_var %v_param_2: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
let %x_1746: Tensor[(1, 64, 256, 256), float32] /* ty=Tensor[(1, 64, 256, 256), float32] */ = nn.bias_add(%x_1745, %v_param_2) /* ty=Tensor[(1, 64, 256, 256), float32] */;
free_var %v_param_3: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
free_var %v_param_4: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
free_var %v_param_5: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
free_var %v_param_6: Tensor[(64), float32] /* ty=Tensor[(64), float32] */;
let %x_1747: (Tensor[(1, 

In [3]:
def get_network():
    img_size = 8
    out_channels = 16
    batch_size = 1
    data = relay.var("data", relay.TensorType((batch_size, 3, img_size, img_size), "float16"))
    dense_weight = relay.var(
        "dweight", relay.TensorType((batch_size, 16 * img_size * img_size), "float16")
    )
    weight = relay.var("weight")
    second_weight = relay.var("second_weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")
    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.nn.batch_flatten(simple_net)
    simple_net = relay.nn.dense(simple_net, dense_weight)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
    data_shape = (batch_size, 3, img_size, img_size)
    net, params = testing.create_workload(simple_net)
    return net, params, data_shape




In [57]:
def get_skip_net():
    img_size = 8
    out_channels = 16
    batch_size = 1
    data = relay.var("data", relay.TensorType((batch_size, 3, img_size, img_size), "float16"))
    dense_weight = relay.var(
        "dweight", relay.TensorType((batch_size, 16 * img_size * img_size), "float16")
    )
    weight = relay.var("weight")
    second_weight = relay.var("second_weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")
    x0 = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    x1 = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.op.concatenate(relay.Tuple([x0, x1]), axis=0)
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.nn.batch_flatten(simple_net)
    simple_net = relay.nn.dense(simple_net, dense_weight)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
    data_shape = (batch_size, 3, img_size, img_size)
    net, params = testing.create_workload(simple_net)
    return net, params, data_shape

print(get_skip_net()[0])

def @main(%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(2, 1), float16] {
  %0 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %2 = (%0, %1) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(1, 16, 8, 8), float16]) */;
  %3 = concatenate(%2) /* ty=Tensor[(2, 16, 8, 8), float16] */;
  %4 = nn.batch_norm(%3, %bn_gamma, %bn_beta, %bn_mean, %bn_

In [63]:
body = get_skip_net()[0]['main'].body
body = run_opt_pass(body, transform.ToANormalForm())
body = run_opt_pass(body, transform.InferType())

In [64]:
body

LetNode(Var(x_51), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6d1fb98), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_52), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c03a68), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_53), Tuple([Var(x_51), Var(x_52)]), LetNode(Var(x_54), CallNode(Op(concatenate), [Var(x_53)], relay.attrs.ConcatenateAttrs(0x6cbe958), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]), LetNode(Var(x_55), CallNode(Op(nn.batch_norm), [Var(x_54), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs

In [ ]:
LetNode(
    Var(x_51), 
    CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6d1fb98), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
    LetNode(
        Var(x_52), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c03a68), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
        LetNode(
            Var(x_53), 
            Tuple([Var(x_51), Var(x_52)]), 
            LetNode(
                Var(x_54), 
                CallNode(Op(concatenate), [Var(x_53)], relay.attrs.ConcatenateAttrs(0x6cbe958), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]), 
                LetNode(
                    Var(x_55), 
                    CallNode(Op(nn.batch_norm), [Var(x_54), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6bfd428), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 
                    LetNode(Var(x_56), TupleGetItemNode(Var(x_55), 0), LetNode(Var(x_57), CallNode(Op(nn.relu), [Var(x_56)], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_58), CallNode(Op(nn.batch_flatten), [Var(x_57)], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_59), CallNode(Op(nn.dense), [Var(x_58), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6bf3428), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)]), Var(x_59))))))))))

In [65]:
LetNode(
    Var(x_42, ty=TensorType([1, 16, 8, 8], float16)), 
    CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c7b508), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
    ), 
    LetNode(
        Var(x_43, ty=TensorType([1, 16, 8, 8], float16)), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c2a268), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
        ), 
        LetNode(
            Var(x_44, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])), 
            Tuple([Var(x_42, ty=TensorType([1, 16, 8, 8], float16)), Var(x_43, ty=TensorType([1, 16, 8, 8], float16))]
            ), 
            LetNode(
                Var(x_45, ty=TensorType([2, 16, 8, 8], float16)), 
                CallNode(Op(concatenate), [Var(x_44, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)]))], relay.attrs.ConcatenateAttrs(0x6c938b8), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]
                ), 
                LetNode(
                    Var(x_46, ty=TupleTypeNode([TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 
                    CallNode(Op(nn.batch_norm), [Var(x_45, ty=TensorType([2, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6cb11b8), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]
                    ), 
                    LetNode(Var(x_47, ty=TensorType([2, 16, 8, 8], float16)), TupleGetItemNode(Var(x_46, ty=TupleTypeNode([TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), LetNode(Var(x_48, ty=TensorType([2, 16, 8, 8], float16)), CallNode(Op(nn.relu), [Var(x_47, ty=TensorType([2, 16, 8, 8], float16))], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_49, ty=TensorType([2, 1024], float16)), CallNode(Op(nn.batch_flatten), [Var(x_48, ty=TensorType([2, 16, 8, 8], float16))], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_50, ty=TensorType([2, 1], float16)), CallNode(Op(nn.dense), [Var(x_49, ty=TensorType([2, 1024], float16)), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6b79f78), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)]), Var(x_50, ty=TensorType([2, 1], float16)))))))))))

NameError: name 'LetNode' is not defined

In [ ]:
[TupleGetItemNode(
    CallNode(Op(nn.batch_norm), 
        [CallNode(Op(concatenate), 
        [Tuple([CallNode(Op(nn.conv2d), 
        [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x6d34998), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x6bfeed8), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)])])], 
        relay.attrs.ConcatenateAttrs(0x6c1cbe8), 
        [TupleTypeNode([TensorType([1, 16, 8, 8], float16), 
        TensorType([1, 16, 8, 8], float16)])]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6c05b58), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([2, 16, 8, 8], float16)])], (nullptr), [TensorType([2, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6c6c018), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)])

In [4]:
net, params, data_shape = get_network()

In [14]:
net['main'].body

CallNode(Op(nn.dense), [CallNode(Op(nn.batch_flatten), [CallNode(Op(nn.relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([1, 16, 8, 8], float16)])], (nullptr), [TensorType([1, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)])

In [ ]:
CallNode(Op(nn.dense), [CallNode(Op(nn.batch_flatten), [CallNode(Op(nn.relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(
    bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([1, 16, 8, 8], float16)])], (nullptr), [TensorType([1, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)])


In [20]:
# lets explore
body = net['main'].body
body = run_opt_pass(body, transform.ToANormalForm())
body = run_opt_pass(body, transform.InferType())
cur = body

In [25]:
print(body)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
let %x_0: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
free_var %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
let %x_1: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_0, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;

In [37]:
cur = body

In [21]:
pre = cur
cur = cur.value
print(cur)
print(type(cur))

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */
<class 'tvm.relay.expr.Call'>


In [39]:
type(cur)

tvm.relay.expr.Let

In [40]:
cur

LetNode(Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), CallNode(Op(nn.batch_norm), [Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), LetNode(Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), 

In [41]:
cur.var

Var(x_0, ty=TensorType([1, 16, 8, 8], float16))

In [42]:
cur.value

CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)])

In [43]:
cur.body

LetNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), CallNode(Op(nn.batch_norm), [Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), LetNode(Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), LetNode(Var(x_3, ty=TensorType([1, 16, 8, 8], float16)), CallNode(Op(nn.relu), [Var(x_2, ty=TensorType([1, 16, 8, 8], float16))], (nullptr), [TensorType([1, 16, 8, 8], float16)]), LetNode(Var(x_4, ty=TensorType([1, 1024], float16)), 

In [44]:
cur.span

In [28]:
LetNode(
    Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), 
    CallNode(
        Op(nn.conv2d), 
        [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x65c2968), 
        [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
    ), 
    LetNode(
        Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 
        CallNode(
            Op(nn.batch_norm), 
            [
                Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), 
                Var(bn_gamma, ty=TensorType([16], float16)), 
                Var(bn_beta, ty=TensorType([16], float16)), 
                Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))
            ], 
            relay.attrs.BatchNormAttrs(0x331cbf8), 
            [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]
        ), 
        LetNode(
            Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), 
            TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), 
            LetNode(
                Var(x_3, ty=TensorType([1, 16, 8, 8], float16)), 
                CallNode(
                    Op(nn.relu), 
                    [Var(x_2, ty=TensorType([1, 16, 8, 8], float16))], 
                    (nullptr), 
                    [TensorType([1, 16, 8, 8], float16)]
                ), 
                LetNode(
                    Var(x_4, ty=TensorType([1, 1024], float16)), 
                    CallNode(
                        Op(nn.batch_flatten), 
                        [Var(x_3, ty=TensorType([1, 16, 8, 8], float16))], 
                        (nullptr), 
                        [TensorType([1, 16, 8, 8], float16)]
                    ), 
                    LetNode(
                        Var(x_5, ty=TensorType([1, 1], float16)), 
                        CallNode(
                            Op(nn.dense), 
                            [Var(x_4, ty=TensorType([1, 1024], float16)), Var(dweight, ty=TensorType([1, 1024], float16))], 
                            relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)]
                        ), 
                        Var(x_5, ty=TensorType([1, 1], float16))
                    )
                )
            )
        )
    )
)


NameError: name 'LetNode' is not defined

In [6]:
def graph_split(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    if (
                        split_conf
                        and split_operator_name in operator_index_map
                        and operator_index_map[split_operator_name] >= split_operator_index
                    ):
                        # Do graph splitting.
                        split_conf.pop(0)
                        snode_dep.append({"nodes": {}, "ref_nodes": {}})
                        ann = _recursion(
                            anf.body,
                            pipeline_mods,
                            split_conf,
                            constant_expr,
                        )
                        snode_dep.pop()
                        dep_vars = get_dep_var(snode_dep)
                        # When the nodes of the current subgraph are the depedency node of another
                        # subgraph, we need to set them as the output of current subgraph.
                        body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                        # when the operator of current subgraph uses previous subgraph constant
                        # as the argument of a "relay.expr.call", such constant may become a free
                        # varaible if the constant does not exist in the current subgraph.
                        # merge the previous constant with current subgraph to avoid such issue.
                        if constant_expr:
                            ann = merge_constant_expr(constant_expr, ann)
                        ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                        mod = tvm.IRModule.from_expr(ann)
                        pipeline_mods.insert(0, mod)
                        # Return the last node of the current subgraph.
                        return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, mod)
    return pipeline_mods


In [7]:
split_config = [{"op_name": "nn.relu", "op_index": 0}]
print(net['main'])
subgraphs = graph_split(net["main"], split_config, params)

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(1, 1), float16] {
  %0 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.batch_norm(%0, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;
  %2 = %1.0 /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %3 = nn.relu(%2) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %4 = nn.batch_flatten(%3) /* ty=Tensor[(1, 1024), float16] */;
  nn.dense(%4, %dw

In [9]:
len(subgraphs)

2

In [21]:
netnet = run_opt_pass(net["main"], transform.ToANormalForm())
netnet = run_opt_pass(netnet, transform.InferType())

In [34]:
print(netnet)

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(1, 1), float16] {
  let %x_12: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  let %x_13: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_12, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8),

In [32]:
print(netnet.body)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
let %x_12: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
free_var %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
let %x_13: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_12, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) 

In [33]:
print(netnet.body.value)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */


In [35]:
print(type(netnet.body.value.args))
print(netnet.body.value.args[0])

<class 'tvm.ir.container.Array'>
free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
%data


In [16]:
subgraphs[0]['main']

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */) {
  %0 = nn.conv2d(%data, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float16] */, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.batch_norm(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float16] */, meta[relay.Constant][2] /* ty=Tensor[(16), float16] */, meta[relay.Constant][3] /* ty=Tensor[(16), float16] */, meta[relay.Constant][4] /* ty=Tensor[(16), float16] */) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;
  %2 = %1.0 /* ty=Tensor[(1, 16, 8, 8), float16] */;
  nn.relu(%2) /* ty=Tensor[(1, 16, 8, 8), float16] */
}



In [14]:
subgraphs[1]['main']

fn (%data_n_0: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */) {
  %0 = nn.batch_flatten(%data_n_0) /* ty=Tensor[(1, 1024), float16] */;
  nn.dense(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 1024), float16] */, units=None) /* ty=Tensor[(1, 1), float16] */
}

In [2]:
model_keras = Model(3, 1, 16)

In [3]:
target = 'cuda'
dev = tvm.cuda()
img_size = 256
input_data = np.random.normal(0,1,(1,img_size,img_size,3)).astype(np.float32)
# tvm result
input_data = input_data.transpose([0, 3, 1, 2])

shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [7]:
mod['main']

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.batch_norm(%1, %v_param_3, %v_param_4, %v_param_5, %v_param_6, epsilon=0.001f);
  %3 = %2.0;
  %4 = nn.leaky_relu(%3, alpha=0.2f);
  %5 = nn.conv2d(%4, %v_param_7, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %6 = nn.bias_add(%5, %v_param_8);
  %7 = nn.batch_norm(%6, %v_param_9, %v_param_10, %v_param_11, %v_param_12, epsilon=0.

In [94]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


ValueError: Traceback (most recent call last):
  7: TVMFuncCall
  6: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  5: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  4: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  3: tvm::relay::backend::RelayBuildModule::OptimizeImpl(tvm::IRModule)
  2: tvm::relay::backend::BindParamsInModule(tvm::IRModule, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> 

In [96]:
z = mm.get_global_vars()[0]
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: <class 'tvm.ir.expr.GlobalVar'> has no attribute params

In [88]:
for func in mm.functions:
    print(func.astext().split('\n')[-1].split('@')[-1])

tvmgen_default_ccompiler_main_4
tvmgen_default_ccompiler_main_0
tvmgen_default_ccompiler_main_2
main


In [46]:
x = relay.var("x", shape=(8, 8))
y = relay.var("y", shape=(8, 8))
z = mm['tvmgen_default_ccompiler_main_0'](x,y)
f = relay.Function([x, y], z)
mod1 = tvm.IRModule()
mod1['main'] = f

In [69]:
type(z)

tvm.relay.function.Function

In [72]:
z = mm['tvmgen_default_ccompiler_main_0']
z.attrs.global_symbol = 'main'
z = z.with_attr("global_symbol", "main")
mod = tvm.IRModule(z)
# mod['main'] = z
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)



TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)>::AssignTypedLambda<tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}>(tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::TVMRetValue)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::runtime::Ma

In [ ]:
# func0 = func0.with_attr("global_symbol", "main")


In [49]:
mod1

#[version = "0.0.5"]
def @main(%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}

In [73]:
z = mm['tvmgen_default_ccompiler_main_0']
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: 'GraphExecutorFactoryModule' object has no attribute 'body'

In [61]:
f

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */) {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
}

In [59]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod2, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [40]:
print(mm['tvmgen_default_ccompiler_main_0'])

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
} /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */


In [39]:
print(f)

fn (%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}


In [15]:
mod = tvm.IRModule()
mod["main"] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [16]:
lib

In [78]:
print(type(legion))


<class 'tvm.relay.function.Function'>


In [80]:
print(type(mm['main']))


<class 'tvm.relay.function.Function'>


In [43]:
# f = type(mm['tvmgen_default_ccompiler_main_0'])
# mod = tvm.IRModule()
# mod["main"] = mm['tvmgen_default_ccompiler_main_0']
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


In [33]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [11]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


ValueError: Traceback (most recent call last):
  8: TVMFuncCall
  7: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  6: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  5: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  4: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::relay::backend::GraphExecutorCodegenModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)

In [65]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)


/home/jd/workspace/tvm-v0.9.0/python/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


In [26]:
lib

In [45]:
lib['get_graph_json']

In [62]:
graph_json_raw = lib['get_graph_json']()


In [19]:
import pygraphviz as pgv
import json
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            # if args.show_size == 1:
            if 1 == 1:
                src_size = 1
                for i in json_data['attrs']['shape'][1][src[0]]:
                    src_size = src_size * i
                
                dst_size = 1
                for i in json_data['attrs']['shape'][1][node_idx]:
                    dst_size = dst_size * i
                # print(src[0], json_data['nodes'][src[0]]['name'], src_size)

                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]) + "[{}]".format(src_size), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]) + "[{}]".format(dst_size))
            else:
                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

In [63]:
show_graph(graph_json_raw, 'test')
